<h3> Importation des librairies et sélection des options d'affichage </h3>

In [6]:
import networkx as nx
import gurobipy
from gurobipy import Model, GRB
import matplotlib.pyplot as plt
import sys
import random
import csv

options = {
    "font_size": 20,
    "node_size": 1000,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 5,
    "width": 5,
    "with_labels": True,
}

Fonction de géneration d'instance de graphique avec sélection de type de graphe

In [7]:
# Generer des instances de graphes

def generate_graph(graph_type, n_nodes, p=None):
    """
    Génère un graphe selon le type spécifié.
    :param graph_type: Type de graphe ('complete', 'cycle', 'random').
    :param n_nodes: Nombre de nœuds.
    :param p: Probabilité pour les graphes aléatoires (nécessaire pour 'random').
    :return: Graphe généré.
    """
    if graph_type == 'complete':
        G = nx.complete_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'cycle':
        G = nx.cycle_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'random':
        G = nx.erdos_renyi_graph(n_nodes, p, directed=True)
    else:
        raise ValueError("Type de graphe non supporté")
    return G


Création des valeurs de capacité et de coût 

In [8]:
def assign_capacities_and_costs(G, capacity_range, cost_range):
    """
    Assigne des capacités et des coûts de routage aux arcs.
    :param G: Graphe.
    :param capacity_range: Intervalle pour les capacités (e.g., (1, 10)).
    :param cost_range: Intervalle pour les coûts (e.g., (1, 5)).
    :return: Graphe avec attributs assignés.
    """
    for u, v in G.edges:
        G[u][v]['capacity'] = random.randint(*capacity_range)
        G[u][v]['cost'] = random.uniform(*cost_range)
    return G

Generation de la demande du traffic 

In [9]:
def generate_traffic_demand(G, S_size, demand_range):
    """
    Génère des sous-ensembles S de nœuds et des demandes de trafic.
    :param G: Graphe.
    :param S_size: Taille de l'ensemble S.
    :param demand_range: Intervalle pour les demandes de trafic (e.g., (1, 20)).
    :return: Ensemble S et vecteur des demandes de trafic.
    """
    nodes = list(G.nodes)
    S = random.sample(nodes, S_size)
    traffic_demand = {}
    for i in S:
        for j in S:
            if i != j:
                traffic_demand[(i, j)] = random.randint(*demand_range)
    return S, traffic_demand

Sauvegarde de l'instance dans un csv

In [39]:
def save_to_csv(G, S, traffic_demand, filename):
    """
    Sauvegarde le graphe, les capacités, les coûts et les demandes de trafic dans un fichier CSV.
    :param G: Graphe.
    :param S: Sous-ensemble des nœuds.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param filename: Nom du fichier CSV.
    """
    with open("graph/"+filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Écriture des arcs avec capacités et coûts
        writer.writerow(['Source', 'Target', 'Capacity', 'Cost'])
        for u, v in G.edges:
            writer.writerow([u, v, G[u][v]['capacity'], G[u][v]['cost']])
        
        # Écriture des demandes de trafic
        writer.writerow([])
        writer.writerow(['entry', 'Target', 'Traffic Demand'])
        for (i, j), demand in traffic_demand.items():
            writer.writerow([i, j, demand])
        
        # Écriture de l'ensemble S
        writer.writerow([])
        writer.writerow(['Subset S'])
        writer.writerow(S)

In [24]:
def create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename):
    G = generate_graph(graph_type, n_nodes, p=0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    save_to_csv(G, S, traffic_demand, filename)

    print(f"Instance générée et sauvegardée dans {filename}")
    
    

Exemple utilisation de géneration, attribution des capacités et coût et de la sauvegarde du graphe 

In [40]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_instance2.csv'  # Nom du fichier CSV

    create_instance(graph_type, n_nodes, capacity_range, cost_range, S_size, demand_range, filename)
    
    

Instance générée et sauvegardée dans graph_instance2.csv


Fonction d'une lecture d'une instance 

In [41]:
def read_instance(filename):
    """
    Lit une instance à partir d'un fichier CSV.
    Args:
        filename (string): nom du fichier csv
    Return:
        Graphe, sous-ensemble S, et demande de trafic  
    """
    G = nx.DiGraph()
    traffic_demand = {}
    S = []
    with open("graph/"+filename, mode='r') as file:
        reader = csv.reader(file)
        section = "arcs"
        for row in reader:
            if not row:
                continue
            if row[0] == "Source":
                continue
            if row[0] == "entry":
                section = "traffic"
            if row[0] == "Subset S":
                section = "subset"
                continue
            if section == "arcs":
                G.add_edge(int(row[0]), int(row[1]), capacity=float(row[2]), cost=float(row[3]))
            elif section =="traffic":
                traffic_demand[(int(row[0]), int(row[1]))] = int(row[2])
            elif section == "subset":
                S = list(map(int, row))
                break
    return G, S, traffic_demand 
    

Fonction pour créer et resoudre un PL pour une fonction objectif donnée 

In [60]:
def solve_linear_program(G, traffic_demand, objective):
    """
    Résout un programme linéaire associé à une fonction objective spécifiée.

    Args:
        G (networkx graph): graphe de l'instance 
        traffic_demand (_type_): valeur de la demande pour le sous ensemble S
        objective (_type_): fonction objective choisie
        
    return:
        solutions de routage et valeur optimale 
    """
    
    model = Model("Routing")
    
    #Variables de décision
    x = {}
    for (i, j), demand in traffic_demand.items():
        for u, v in G.edges:
            x[i, j, u, v] = model.addVar(lb=0, ub=1, name=f"x_{i}_{j}_{u}_{v}")
    
    #Contraintes de conservation de flux
    for (i, j), demand in traffic_demand.items():
        for node in G.nodes:
            inflow = sum(x[(i, j, u, v)] for u, v in G.in_edges(node))
            outflow = sum(x[(i, j, u, v)] for u, v, in G.out_edges(node))
            if node == i:
                model.addConstr(outflow - inflow == 1, name=f"flow_src_{i}_{j}_{node}")
            elif node == j:
                model.addConstr(inflow - outflow == 1, name=f"flow_dst_{i}_{j}_{node}")
            else:
                model.addConstr(inflow - outflow == 0, name=f"flow_mid_{i}_{j}_{node}")

    # Constrainte de capacité 
    for u, v in G.edges:    
        model.addConstr(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) <= G[u][v]['capacity'],
            name=f"capacity_{u}_{v}"
        )
    
    # Objectifs
    if objective == "cost":
        model.setObjective(
            sum(x[(i, j, u, v)] * demand * G[u][v]['cost'] for (i, j), demand in traffic_demand.items() for u, v in G.edges),
            GRB.MINIMIZE
        )
    elif objective == "max_utlilization":
        utilisation = model.addVar(lb=0, ub=GRB.INFINITY, name="max_utilisation")
        for u, v in G.edges:
            model.addConstr(
                utilisation >=
                sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity'],
                name=f"utilisation_{u}_{v}"
            )
        model.setObjective(utilisation, GRB.MINIMIZE)
    elif objective == "avg_utilisation":
        avg_utilisation = sum(
            sum(demand * x[(i, j, u, v)] for (i, j), demand in traffic_demand.items()) / G[u][v]['capacity']
            for u, v in G.edges
        ) / G.number_of_edges()
        model.setObjective(avg_utilisation, GRB.MINIMIZE)
    else:
        raise ValueError("Fonction objectif non reconnue")

    #Résolution
    model.optimize()
    
    # Débogage en cas d'infaisabilité
    if model.status == GRB.INFEASIBLE:
        print("Modèle infaisable. Analyse des conflits...")
        model.computeIIS()
        model.write("infeasible.ilp")
        print("Conflits enregistrés dans infeasible.ilp")
        return None
    
    #Extraction de la solution
    if model.status == GRB.OPTIMAL:
        solution = {key: var.x for key, var in x.items() if var.x > 0}
        return solution, model.ObjVal
    else:
        raise Exception("Pas de solution optimale trouvée")
    

Fonction de test de la résolution d'une instance de graphe

In [58]:
def test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective):
    """
    Teste le programme de résolution de PL sur un type de graphe

    Args:
        graph_type (string): type de graphe 
        n_nodes (int): nombre de node dans V
        S_size (int): nombre de node dans S
        demand_range (tuple): tuple de 2 int pour le range des valeurs de demande 
        capacity_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        cost_range (tuple): tuple de 2 int pour le range des valeurs de capacité
        objective (string): nom de la fonction objective choisi (cost, max_utlilization, avg_utlilization)
    """
    G = generate_graph(graph_type, n_nodes, 0.3)
    G = assign_capacities_and_costs(G, capacity_range, cost_range)
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)
    solution, optimal_value = solve_linear_program(G, traffic_demand, objective)
    print(f"Solution optimale pour {objective}:", solution)
    print(f"Valeur optimale pour {objective}: {optimal_value}")

In [61]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 15  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_instance.csv'  # Nom du fichier CSV
    objective = "cost" # Choix de la fonction objective 
    test_program(graph_type, n_nodes, S_size, demand_range, capacity_range, cost_range, objective)
    
    

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 361 rows, 1220 columns and 3660 nonzeros
Model fingerprint: 0xac6c063f
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
Modèle infaisable. Analyse des conflits...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.800000e+01   

TypeError: cannot unpack non-iterable NoneType object